In [ ]:
import copy
import numpy as np
from lmfit import Parameters
from lmfit import minimize, fit_report

In [ ]:
import pyroll.core as pr
import pyroll.ring_model_thermal
from pyroll.freiberg_flow_stress import flow_stress

In [ ]:
measured_temperature_pyr_1 = 956 + 273.15
measured_temperatures_pyr_2 = np.array([833 + 273.15])
root_mean_square_temperature = np.sqrt(np.mean(measured_temperatures_pyr_2 ** 2))

In [ ]:
in_profile = pr.Profile.from_groove(
    groove=pr.FalseRoundGroove(
        depth=8.896e-3,
        r1=2e-3,
        r2=10.9e-3,
        flank_angle=57.5
    ),
    material = ["BST500", "steel"],
    strain=0,
    temperature=measured_temperature_pyr_1,
    gap=4.6e-3,
    filling=1,
    velocity=7
)


density = pd.read_csv(filepath / "in_profile" / "bst500_density.csv").density
density_temperature = pd.read_csv(filepath / "in_profile" / "bst500_density.csv").kelvin_temperature
specific_heat_capacity_real = pd.read_csv(
    filepath / "in_profile" / "bst500_specific_heat_capacity_real_measurement.csv").specific_heat_capacity * 1000
specific_heat_capacity_temperature_real = pd.read_csv(
    filepath / "in_profile" / "bst500_specific_heat_capacity_real_measurement.csv").celsius_temperature + 273.15
specific_heat_capacity = pd.read_csv(
    filepath / "in_profile" / "bst500_specific_heat_capacity.csv").specific_heat_capacity * 1000
specific_heat_capacity_temperature = pd.read_csv(
    filepath / "in_profile" / "bst500_specific_heat_capacity.csv").celsius_temperature + 273.15
thermal_conductivity = pd.read_csv(filepath / "in_profile" / "bst500_thermal_conductivity.csv").thermal_conductivity
thermal_conductivity_temperature = pd.read_csv(
    filepath / "in_profile" / "bst500_thermal_conductivity.csv").celsius_temperature + 273.15

density_function = interp1d(density_temperature, density)
specific_heat_capacity_function = interp1d(specific_heat_capacity_temperature, specific_heat_capacity)
thermal_conductivity_function = interp1d(thermal_conductivity_temperature, thermal_conductivity)


@pr.Profile.density
def density(self: pr.Profile):
    return density_function(self.temperature)


@pr.Profile.specific_heat_capacity
def specific_heat_capacity(self: pr.Profile):
    return specific_heat_capacity_function(self.temperature)


@pr.Profile.thermal_conductivity
def thermal_conductivity(self: pr.Profile):
    return thermal_conductivity_function(self.temperature)

In [ ]:
sequence = pr.PassSequence([])


def solve_sequence_for_errors(params):
    def htc_wehage(params, surface_temperature, velocity_ratio, diameter_ratio):
        c1 = params["c1"].value
        c2 = params["c2"].value
        c3 = params["c3"].value
        c4 = params["c4"].value
        c5 = params["c5"].value
        c6 = params["c6"].value
        c7 = params["c7"].value

        htc_surface_temperature = c1 + c2 * 1e6 / surface_temperature
        htc_velocities = c3 * velocity_ratio ** c4
        htc_water_volume_flux = c5 * volume_flux ** c6 * diameter_ratio ** c7

        return htc_surface_temperature * htc_velocities * htc_water_volume_flux

    copied_sequence = copy.deepcopy(sequence)
    copied_in_profile = copy.deepcopy(in_profile)

    for cooling_pipe in copied_sequence.cooling_pipes:
        velocity_ratio = 0
        diameter_ratio = 0
        for de in cooling_pipe.disk_elements:
            mean_surface_temperature = (de.in_profile.surface_temperature + de.out_profile.surface_temperature )/ 2
            cooling_pipe.heat_transfer_coefficient = htc_wehage(params, mean_surface_temperature, velocity_ratio, diameter_ratio)

    copied_sequence.solve(copied_in_profile)

    error = copied_sequence[-1].out_profile.surface_temperature - root_mean_square_temperature

    return error


In [ ]:
params_wehage_model = Parameters()
params_wehage_model.add('c1', value=1e-5, min=1e-20, max=1e-1, vary=True)
params_wehage_model.add('c2', value=1e-5, min=1e-20, max=1e-1, vary=True)
params_wehage_model.add('c3', value=1e-5, min=1e-20, max=1e-1, vary=True)
params_wehage_model.add('c4', value=1e-5, min=1e-20, max=1e-1, vary=True)
params_wehage_model.add('c5', value=1e-5, min=1e-20, max=1e-1, vary=True)
params_wehage_model.add('c6', value=1e-5, min=1e-20, max=1e-1, vary=True)
params_wehage_model.add('c7', value=1e-5, min=1e-20, max=1e-1, vary=True)

In [ ]:
result_wehage_model_cobyla = minimize(
    solve_sequence_for_errors,
    params_wehage_model,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

In [ ]:
print(fit_report(result_wehage_model_cobyla))